In [1]:
# Start the pyspark session

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import re

spark = SparkSession.builder.getOrCreate()

## Data Acquisition
These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.
2. Let's see **how writing to the local disk works in spark**:
    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

### 1. Read the case, department, and source data into their own spark dataframes.

In [82]:
# Read case.csv

df_case = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True)
df_case

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]

In [3]:
# Read dept.csv

df_source = (spark.read.format("csv")
             .option("sep", ",")
             .option("inferSchema", True)
             .option("header", True)
             .load("data/source.csv")
            )

df_source

DataFrame[source_id: string, source_username: string]

In [4]:
# Read dept.csv

df_dept = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True)
df_dept

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

### 2. Write to the local disk works in spark

In [5]:
# # Write the df_source spark dataframe to csv

# df_source.write.csv("source_csv", mode="overwrite")
# df_source.write.json("source_json", mode="overwrite")

**Takeaways**
1. Two folders are created respectively, instead of .csv and .json files.
2. Inside each folder, there are two files: _SUCCESS and part-***** files.

### 3.  Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [6]:
# Inspect the df_case

df_case.printSchema()
df_case.show(1, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  


In [83]:
# Rename column SLA_due_date to case_due_date
df_case = df_case.withColumnRenamed("SLA_due_date", "case_due_date")

In [84]:
# Handle dtypes: 
    # convert the Yes/No values to boolean using expr() method
    # cast column council_district to integer

df_case = (
    df_case.withColumn("case_late", expr("case_late == 'YES'"))
    .withColumn("case_closed", expr("case_closed == 'YES'"))
    .withColumn("council_district", format_string("%03d", col("council_district")))
)

df_case = df_case.withColumn("council_district", df_case.council_district.cast("int"))

In [85]:
# Handle dates
    # Convert the string datetime to standard format

# Define the format
fmt = "M/d/yy H:mm"

# Use to_timestamp method to standardize the datetime

df_case = (
    df_case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))

)

In [86]:
# Add features

# Compute the max date
max_date = df_case.select(max("case_closed_date")).first()[0]
print(max_date)
print(type(max_date))

df_case = (
    df_case.withColumn("num_weeks_late", expr("num_days_late / 7 AS num_weeks_late"))
    .withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))
    .withColumn("case_age", datediff(lit(max_date), "case_opened_date"))
    .withColumn("days_to_closed", datediff("case_closed_date", "case_opened_date"))
    .withColumn("case_lifetime", 
                when(expr("!case_closed"), col("case_age")).otherwise(col("days_to_closed"))
               )
)

2018-08-08 10:38:00
<class 'datetime.datetime'>


In [ ]:
def zipcode(string):
    regexp = r'\d+$'
    return re.search(regexp, string)

In [ ]:
zipcode("2315  EL PASO ST, San Antonio, 78207")

In [87]:
# Join dept.csv

df = (
    df_case.join(df_dept, "dept_division", "left")
#     .drop(df_dept.dept_name)
#     .drop(df_case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [88]:
# Join df_source

df = df.join(df_source, "source_id", "left")

In [89]:
df.count()

855269

In [14]:
df.printSchema()
df.show(1, vertical=True)

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)
 |-

In [90]:
# In case we want to querry our dataframe with Spark SQL

df.createOrReplaceTempView('df')

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [16]:
row = (
    df.filter(col("case_closed") == False) # Currently open issue
    .sort(col("case_opened_date").desc())  # the latest issue
    .first()
)

row.case_opened_date

datetime.datetime(2018, 8, 8, 10, 38)

In [18]:
# Use spark SQL to answer this question

spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 5
''').show()

+-------------+
|days_past_due|
+-------------+
|         1416|
|         1416|
|         1416|
|         1415|
|         1413|
+-------------+



### 2. How many Stray Animal cases are there?

In [19]:
# Return the number of rows with Stray Animal cases

df.filter(df_case.service_request_type == "Stray Animal").count()

27361

In [22]:
# Using spark SQL

spark.sql('''
SELECT count(*)
FROM df
WHERE service_request_type == "Stray Animal"
''').show()

+--------+
|count(1)|
+--------+
|   27361|
+--------+



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [23]:
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .where(expr('''service_request_type != "Officer Standby"'''))
    .count()
)

116295

In [ ]:
# Using spark sql

spark.sql('''
SELECT count(*)
FROM df
WHERE service_request_type != "Officer Standby"
AND dept_division == "Field Operations"
''').show()

### 4. Convert the council_district column to a string column.

In [24]:
# Use cast() method

df.withColumn("council_district", col("council_district").cast("int"))

DataFrame[source_id: string, dept_division: string, case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, service_request_type: string, SLA_days: double, case_status: string, request_address: string, council_district: int, num_weeks_late: double, zipcode: string, case_age: int, days_to_closed: int, case_lifetime: int, dept_name: string, department: string, dept_subject_to_SLA: boolean, source_username: string]

### 5. Extract the year from the case_closed_date column.

In [25]:
df.select(
    "case_closed_date",
    year("case_closed_date")
).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [26]:
df.withColumn(
    "num_hours_late",
    df.num_days_late * 24
).select(
    "num_hours_late",
    "num_days_late"    
).show(5)

+-------------------+-------------------+
|     num_hours_late|      num_days_late|
+-------------------+-------------------+
|     -23964.2102784| -998.5087616000001|
|-48.302500007999996|-2.0126041669999997|
|      -72.536111112|       -3.022337963|
|      -360.27555552|       -15.01148148|
|  8.931944448000001|0.37216435200000003|
+-------------------+-------------------+
only showing top 5 rows



### 7. Join the case data with the source and department data.

In [27]:
# Already did

### 8. Are there any cases that do not have a request source?

In [32]:
# Using spark sql

spark.sql('''
select count(*)
from df
where source_id == null
''').show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [33]:
# Explain the process

spark.sql('''
select count(*)
from df
where source_id == null
''').explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition, true, [id=#1015]
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(1) LocalTableScan <empty>




In [36]:
# Use spark dataframe

(
    df.select(df.source_id.isNull().cast('int').alias("is_null"))
    .agg(sum('is_null'))
).show()

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [37]:
# Explain

(
    df.select(df.source_id.isNull().cast('int').alias("is_null"))
    .agg(sum('is_null'))
).explain()

== Physical Plan ==
*(4) HashAggregate(keys=[], functions=[sum(cast(is_null#1042 as bigint))])
+- Exchange SinglePartition, true, [id=#1302]
   +- *(3) HashAggregate(keys=[], functions=[partial_sum(cast(is_null#1042 as bigint))])
      +- *(3) Project [cast(isnull(source_id#27) as int) AS is_null#1042]
         +- *(3) BroadcastHashJoin [source_id#27], [source_id#60], LeftOuter, BuildRight
            :- *(3) Project [source_id#27]
            :  +- *(3) BroadcastHashJoin [dept_division#23], [dept_division#80], LeftOuter, BuildRight
            :     :- FileScan csv [dept_division#23,source_id#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/mms3-pro/codeup-data-science/spark-exercises/data/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<dept_division:string,source_id:string>
            :     +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true])), [id=#1288]
            :        +- *(1) Project [de

### 9. What are the top 10 service request types in terms of number of requests?

In [73]:
# Use spark sql

spark.sql('''
select service_request_type, count(*) as num_request
from df
group by service_request_type
order by num_request desc
''').show(10, truncate=False)

+--------------------------------+-----------+
|service_request_type            |num_request|
+--------------------------------+-----------+
|No Pickup                       |89210      |
|Overgrown Yard/Trash            |66403      |
|Bandit Signs                    |32968      |
|Damaged Cart                    |31163      |
|Front Or Side Yard Parking      |28920      |
|Stray Animal                    |27361      |
|Aggressive Animal(Non-Critical) |25492      |
|Cart Exchange Request           |22608      |
|Junk Vehicle On Private Property|21649      |
|Pot Hole Repair                 |20827      |
+--------------------------------+-----------+
only showing top 10 rows



In [51]:
spark.sql('''
select service_request_type, count(*) as num_request
from df
group by service_request_type
order by num_request desc
''').explain()

== Physical Plan ==
*(5) Sort [num_request#1289L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(num_request#1289L DESC NULLS LAST, 200), true, [id=#2205]
   +- *(4) HashAggregate(keys=[service_request_type#24], functions=[count(1)])
      +- Exchange hashpartitioning(service_request_type#24, 200), true, [id=#2201]
         +- *(3) HashAggregate(keys=[service_request_type#24], functions=[partial_count(1)])
            +- *(3) Project [service_request_type#24]
               +- *(3) BroadcastHashJoin [source_id#27], [source_id#60], LeftOuter, BuildRight
                  :- *(3) Project [service_request_type#24, source_id#27]
                  :  +- *(3) BroadcastHashJoin [dept_division#23], [dept_division#80], LeftOuter, BuildRight
                  :     :- FileScan csv [dept_division#23,service_request_type#24,source_id#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/mms3-pro/codeup-data-science/spark-exercises/data/case.csv], Partiti

In [72]:
# Use spark dataframe

(
    df.groupby("service_request_type")
    .count()
    .sort(col("count").desc())
).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



In [52]:

(
    df.groupby("service_request_type")
    .count()
    .sort(col("count").desc())
).explain()

== Physical Plan ==
*(5) Sort [count#1320L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(count#1320L DESC NULLS LAST, 200), true, [id=#2292]
   +- *(4) HashAggregate(keys=[service_request_type#24], functions=[count(1)])
      +- Exchange hashpartitioning(service_request_type#24, 200), true, [id=#2288]
         +- *(3) HashAggregate(keys=[service_request_type#24], functions=[partial_count(1)])
            +- *(3) Project [service_request_type#24]
               +- *(3) BroadcastHashJoin [source_id#27], [source_id#60], LeftOuter, BuildRight
                  :- *(3) Project [service_request_type#24, source_id#27]
                  :  +- *(3) BroadcastHashJoin [dept_division#23], [dept_division#80], LeftOuter, BuildRight
                  :     :- FileScan csv [dept_division#23,service_request_type#24,source_id#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/mms3-pro/codeup-data-science/spark-exercises/data/case.csv], PartitionFilters: [

### 10. What are the top 10 service request types in terms of average days late?

In [92]:
spark.sql('''
select service_request_type, avg(num_days_late) as avg_days_late
from df
where case_late == True
group by service_request_type
order by avg_days_late desc
''').show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg_days_late     |
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [101]:
# Use spark sql

spark.sql('''
select dept_name, avg(num_days_late) as avg_days_late
from df
where case_late == True
group by dept_name
order by avg_days_late
''').show(10, truncate=False)

+-------------------------+------------------+
|dept_name                |avg_days_late     |
+-------------------------+------------------+
|Metro Health             |6.494699602827871 |
|Solid Waste Management   |7.1471727895574135|
|Trans & Cap Improvements |10.662950455078674|
|Parks and Recreation     |22.427807192724135|
|Animal Care Services     |23.4467296347382  |
|Code Enforcement Services|48.085502375106884|
|Development Services     |67.22248485687263 |
|Customer Service         |88.18248182589822 |
|null                     |210.89201994318182|
+-------------------------+------------------+



In [111]:
# Use spark dataframe

(
    df.filter("case_late")
    .groupby("dept_name")
    .agg(mean("num_days_late").alias("avg_days_late"))
    .sort("avg_days_late")
    .withColumn("avg_days_late", round(col("avg_days_late"), 1))
).show(truncate=False)

+-------------------------+-------------+
|dept_name                |avg_days_late|
+-------------------------+-------------+
|Metro Health             |6.5          |
|Solid Waste Management   |7.1          |
|Trans & Cap Improvements |10.7         |
|Parks and Recreation     |22.4         |
|Animal Care Services     |23.4         |
|Code Enforcement Services|48.1         |
|Development Services     |67.2         |
|Customer Service         |88.2         |
|null                     |210.9        |
+-------------------------+-------------+



### 12. How do number of days late depend on department and request type?

In [117]:
# Using spark sql

spark.sql('''
select dept_name, service_request_type, avg(num_days_late) as avg_days_late
from df
where case_closed == True
group by dept_name, service_request_type
order by avg_days_late
''').show(5, truncate=False)

+------------------------+--------------------------------------+-------------------+
|dept_name               |service_request_type                  |avg_days_late      |
+------------------------+--------------------------------------+-------------------+
|City Council            |Request for Research/Information      |null               |
|Trans & Cap Improvements|Engineering Design                    |-1413.83706        |
|Trans & Cap Improvements|Signal Timing Modification By Engineer|-1351.9919107954547|
|Animal Care Services    |Stray Animal                          |-998.8086340218391 |
|Parks and Recreation    |Major Park Improvement Install        |-278.26410401191515|
+------------------------+--------------------------------------+-------------------+
only showing top 5 rows



In [118]:
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("avg_days_late"), count("*").alias("n_cases"))
    .withColumn("avg_days_late", round(col("avg_days_late"), 1))
    .sort(asc("avg_days_late"))
    .show(5, truncate=False)
)

+------------------------+--------------------------------------+-------------+-------+
|dept_name               |service_request_type                  |avg_days_late|n_cases|
+------------------------+--------------------------------------+-------------+-------+
|City Council            |Request for Research/Information      |null         |5      |
|Trans & Cap Improvements|Engineering Design                    |-1413.8      |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer|-1352.0      |22     |
|Animal Care Services    |Stray Animal                          |-998.8       |27346  |
|Parks and Recreation    |Major Park Improvement Install        |-278.3       |271    |
+------------------------+--------------------------------------+-------------+-------+
only showing top 5 rows

